<script type="text/x-mathjax-config">
MathJax.Hub.Config({
  TeX: { equationNumbers: { autoNumber: "AMS" } }
});
</script>

Trust Region Policy Optimization - notes
===========================================

Preliminaries
------------------------------------------
Consider an infinite-horizon discounted Markov decision process (MDP), 
defined by the tuple $(S,A,P,r,\rho_0,\gamma)$
 - S is a finite set of states, 
 - A is a finite set of actions, 
 - $P : S \times A \times S \to R$ is the transition probability distribution,
 - $r : S \to R$ is the reward function
 - $\rho_0 : S \to R$ is the distribution of the initial state $s_0$,
 - $\gamma \in (0, 1)$ is the discount factor.

---

Let 
 - **$\pi$** denote a **stochastic policy** $\pi : S \times A \to [0,1]$, 
 - $\eta(\pi)$ denote its **expected discounted reward**: $\eta(\pi) = E_{s_0,a_0, ...}\bigg[\sum^\infty_{t=0}\gamma^tr(s_t)\bigg]$ where $s_0 \sim \rho_0(s_0)$, $a_t \sim \pi(a_t|s_t)$, $s_{t+1} \sim P(s_{t+1}|s_t, a_t)$.

---

For **state-action value function** $Q_{\pi}$, the **value function** $V_{\pi}$, and the **advantage function** $A_{\pi}$ we will use these standard definitions:
 - $Q_{\pi}(s_t,a_t) = E_{s_{t+1},a_{t+1},...} \bigg[\sum^\infty_{l=0}\gamma^lr(s_{t+l})\bigg]$
 - $V_{\pi}(s_t) = E_{a_{t},s_{t+1}, ...} \bigg[\sum^\infty_{l=0}\gamma^lr(s_{t+l})\bigg]$
 - $A_{\pi}(s,a) = Q_{\pi}(s,a) - V_{\pi}(s)$, where $a \sim \pi(a |s )$, $s \sim P(s |s ,a )$ for $t \geq 0$
 
 ---

**Expected return of another policy $\tilde{\pi}$ in terms of the advantage over $\pi$**, accumulated over timesteps is defined as 

$$
\tag{1}
\eta(\tilde{\pi}) = \eta(\pi)+E_{s_0, a_0, ... \sim \tilde{\pi}} \bigg[\sum_{t=0}^\infty\gamma^tA_{\pi}(s_t,a_t)\bigg]
$$
where $E_{s_0, a_0, ... \sim \tilde{\pi}}[...]$ indicates that actions are sampled $a_t \sim \tilde{\pi}(·|s_t)$.

[proof in *Kakade & Langford - Approximately optimal approximate reinforcement learning. In ICML, volume 2, pp. 267–274, 2002*]

Convergence theory
---


Let $\rho_\pi$ be the **(unnormalized) discounted visitation frequencies**
$$\rho_\pi(s) = P(s_0 = s) + \gamma P(s_1 = s)+\gamma^2P(s_2 = s)+...,$$
where $s_0 \sim \rho_0$ and the actions are chosen according to $\pi$.

Equation $(1)$ can be rewritten to sum over states instead of timesteps if we ues visitation frequencies defined above.

$$
\tag{2}
\eta(\tilde{\pi}) = \eta(\pi) + \sum_s \rho_{\tilde{\pi}}(s) \sum_a \tilde{\pi}(a|s)A_\pi(s,a)
$$
From this equation its easy to see that any policu update $\pi \to \tilde{\pi}$ where $\forall s$, $E[A_\pi(s,a)] > 0$ is guaranteed to increase policy performance $\eta$, or leave it constant if $\forall s$, $E[A_\pi(s,a)] = 0$. 

---

However, in the approximate setting, it will typically be unavoidable, due to estimation and approximation error, that there will be some states $s$ for which the expected advantage is negative, $\sum_a \tilde{\pi}(a|s)A_\pi(s,a)<0$. Due to dependency of $\rho_{\tilde{\pi}}(s)$ on $\tilde{\pi}$ its hard to optimize equation $(2)$. Instead we will introduce local approximation by estimating visitation frequency of new policy by old policy visitation frequency. This is much easier to optimize than last expression.

$$
\tag{3}
L_\pi(\tilde{\pi}) = \eta(\pi) + \sum_s \rho_{\pi}(s) \sum_a \tilde{\pi}(a|s)A_\pi(s,a)
$$

Please note that if we have parametrized policy $\pi_\theta$, where $\pi_\theta(a|s)$ is differentiable function of parameters $\theta$ than $L_\pi$ matches $\eta$ to first order.

$$
L_{\pi_\theta} = \eta(\pi_\theta)
$$
$$
\tag{4}
\nabla_\theta L_{\pi_\theta} \approx \nabla_\theta \eta(\pi_\theta)
$$

[proof in *Kakade & Langford - Approximately optimal approximate reinforcement learning. In ICML, volume 2, pp. 267–274, 2002*]

---

From equation $(4)$ we can conclude that if we improve policy $\pi_\theta$ by taking a small step in the direction of gradient we will also improve $\eta$. However it does not say how big step we can take.

To address this issue Kakade and Langford proposed a policy updating scheme called conservative policy iteration,
for which they could provide explicit lower bounds on the improvement of $\eta$. To define the conservative policy
iteration update, let $\pi_{old}$ denote the current policy, and let $\pi' = argmin_{\pi'} L_{\pi_{old}}(\pi')$ The new policy $\pi_{new}$ was defined to be the following mixture: 

$$
\tag{5}
\pi_{new} = (1-\alpha)\pi_{old}(a|s) + \alpha\pi'(a|s)
$$ 

Kakade and Langford also proved the following result for this update $$\eta(\pi_{new}) \geq L_{\pi_{old}}(\pi_{new})  - \frac{2\epsilon\gamma}{(1-\gamma)^2}\alpha^2$$
Note, however, that so far this bound only applies to mixture policies generated by Equation (5). This policy class is unwieldy and restrictive in practice, and it is desirable for a practical policy update scheme to be applicable to all general stochastic policy classes.



Algorithm
------

1. Use the single path or vine procedures to collect a set of state-action pairs along with Monte Carlo estimates of their Q-values.
2. By averaging over samples, construct the estimated objective and constraint in $$ \text{maximize}_\theta E_{s\sim\rho_{\theta_{old}}, a\sim q} \big[ \frac{\pi_\theta(a|s)}{q(a|s)}Q_{\theta_{old}}(s,a) \big]$$ $$\text{subject to } E_{s\sim\rho_{\theta_{old}}}\big[ D_{KL}(\pi_{\theta_{old}}(\cdot|s) \text{ || } \pi_{\theta}(\cdot|s) \big] \leq \delta
$$
<!---\infdiv{P}{Q} latex formula? --->
3. Approximately solve this constrained optimization problem to update the policy’s parameter vector $\theta$. We use the conjugate gradient algorithm followed by a line search, which is altogether only slightly more expensive than computing the gradient itself.

---

1. First we will be collecting trajectories so we can estimate their Q-values. So what exactly is trajectory?

    **Trajectory** is a sequence of states, actions and rewards. $$traj = s_t, a_t, r_t, s_{t+1}, ...$$ where $t$ is time in our environment, state $s_t$ is state of the environment and $a_t$ is action after which performance we receive reward $r_t$. Previously performed action $a_t$ at state $s_t$ takes us to new state $s_{t+1}$. 
    So performing actions and recieving rewards creates trajectory. With this approach we collect multiple trajectories from which we can estimate Q-values of different state. This is also called Monte Carlo estimate.
    
    lal
    
2. $$ \text{maximize}_\theta E_{s\sim\rho_{\theta_{old}}, a\sim q} \big[ \frac{\pi_\theta(a|s)}{q(a|s)}Q_{\theta_{old}}(s,a) \big]$$ $$\text{subject to } E_{s\sim\rho_{\theta_{old}}}\big[ D_{KL}(\pi_{\theta_{old}}(\cdot|s) \text{ || } \pi_{\theta}(\cdot|s)$$

3. TODO

In [43]:
import numpy as np

def gaussian_coarse_coding(x, start=-.2, end=.2, sigma = 50, n_gaussians=10):
    sep = (end - start) / (n_gaussians - 1)
    return [gauss_pdf(x, (start + i * sep), sigma) for i in range(n_gaussians)]

def gauss_pdf(x, mean, sigma):
    return np.exp(-(x - mean)**2 / 2*sigma**2)


gaussian_coarse_coding(0, -.2, .2, 10, 20)

[0.13533528323661262,
 0.20167293383369225,
 0.2874985690076301,
 0.39208050568087388,
 0.5115243391746761,
 0.63842347455449389,
 0.76225956558820895,
 0.8706596335622917,
 0.95136118285281313,
 0.99447515221388549,
 0.99447515221388549,
 0.95136118285281324,
 0.87065963356229192,
 0.76225956558820895,
 0.63842347455449422,
 0.51152433917467632,
 0.39208050568087399,
 0.28749856900763021,
 0.20167293383369225,
 0.13533528323661281]